In [4]:
emotion_values = {
    "happy": (0.85, 0.75),  # Joy/Happiness
    "angry": (-0.443, 0.908),  # Anger
    "surprised": (0.0, 0.85),  # Surprise
    "sad": (-0.85, -0.35),  # Sadness
    "neutral": (0.0, 0.0),  # Neutral
    "fear": (0.181, 0.949),  # Fear
    "disgusted": (-0.8, 0.5),  # Disgust
    "contempt": (0.307, 0.535),  # Contempt
    "calm": (0.65, -0.5),  # Calmness
    "excited": (0.9, 0.9),  # Excitement
    "bored": (-0.6, -0.9),  # Boredom
    "confused": (-0.3, 0.4),  # Confusion
    "anxious": (-0.85, 0.9),  # Anxiety
    "confident": (0.7, 0.4),  # Confidence
    "frustrated": (-0.8, 0.6),  # Frustration
    "amused": (0.7, 0.5),  # Amusement
    "proud": (0.8, 0.4),  # Pride
    "ashamed": (-0.8, -0.3),  # Shame
    "grateful": (0.7, 0.2),  # Gratitude
    "jealous": (-0.7, 0.5),  # Jealousy
    "hopeful": (0.7, 0.3),  # Hope
    "disappointed": (-0.7, -0.3),  # Disappointment
    "curious": (0.5, 0.5),  # Curiosity
    "overwhelmed": (-0.6, 0.8),  # Overwhelm
    # Add more emotions as needed
}
import torch


def create_emotion_list(emotion_states, total_frames, emotion_values, accentuate=False):
    if accentuate:
        accentuated_values = {
            k: (max(min(v[0] * 1.5, 1), -1), max(min(v[1] * 1.5, 1), -1)) for k, v in emotion_values.items()
        }
        accentuated_values["neutral"] = (0.0, 0.0)  # Keep neutral as is
        emotion_values = accentuated_values

    if len(emotion_states) == 1:
        v, a = emotion_values[emotion_states[0]]
        valence = [v] * (total_frames + 2)
        arousal = [a] * (total_frames + 2)
    else:
        frames_per_transition = total_frames // (len(emotion_states) - 1)

        valence = []
        arousal = []

        for i in range(len(emotion_states) - 1):
            start_v, start_a = emotion_values[emotion_states[i]]
            end_v, end_a = emotion_values[emotion_states[i + 1]]

            v_values = np.linspace(start_v, end_v, frames_per_transition)
            a_values = np.linspace(start_a, end_a, frames_per_transition)

            valence.extend(v_values)
            arousal.extend(a_values)

        valence = valence[:total_frames]
        arousal = arousal[:total_frames]
        # Save valence and arousal as numpy arrays
        valence = np.array(valence)
        arousal = np.array(arousal)

    return (torch.tensor(valence), torch.tensor(arousal), torch.zeros(total_frames))


valence, arousal, zero = create_emotion_list(
    ["surprised", "angry", "calm", "sad", "happy"], 25 * 45, emotion_values, accentuate=True
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter, PillowWriter

# Example valence and arousal values
valence = np.load("../valence.npy")
arousal = np.load("../arousal.npy")

# Create the figure and axis
fig, ax = plt.subplots(subplot_kw={"polar": True})
(arrow,) = ax.plot([], [], marker="o", color="r")  # Arrow marker


# Initialization function
def init():
    ax.set_ylim(0, 1)  # Set the radial limits
    return (arrow,)


# Animation update function
def update(frame):
    theta = (valence[frame] + 1) * np.pi  # Map valence to angle
    r = arousal[frame]  # Use arousal as the radius
    arrow.set_data([theta], [r])
    return (arrow,)


# Create the animation
ani = FuncAnimation(fig, update, frames=len(valence), init_func=init, blit=True)

# Save animation
writer = FFMpegWriter(fps=25)
ani.save("emotion_animation.mp4", writer=writer)

plt.show()